In [36]:
import pandas as pd
import mailparser
import os

# Path to the Enron dataset CSV
csv_path = "./data/emails.csv"

# Check if the dataset exists
if not os.path.exists(csv_path):
    print(
        "Enron dataset not found. Please run enron_processor.py first to download the dataset."
    )
else:
    # Load the dataset
    df = pd.read_csv(csv_path, delimiter=",")

    # Find the specific file "allen-p/all_documents/756."
    target_file = "allen-p/all_documents/756."
    email_row = df[df["file"] == target_file]

    print("EMAIL ROW")
    print(email_row)
    print("DONE")

    if email_row.empty:
        print(f"File '{target_file}' not found in the dataset.")
    else:
        # Parse the email
        try:
            email_message = email_row.iloc[0]["message"]
            mail = mailparser.parse_from_string(email_message)

            # Display email details
            print(f"File: {target_file}")
            print(f"Message ID: {mail.message_id}")
            print(f"Subject: {mail.subject}")
            print(f"From: {mail.from_[0][1] if mail.from_ else 'N/A'}")
            print(f"To: {', '.join([t[1] for t in mail.to]) if mail.to else 'N/A'}")
            print(f"Date: {mail.date}")
            print("\nBody:")
            print(mail.body)
        except Exception as e:
            print(f"Error parsing email: {str(e)}")


EMAIL ROW
                            file  \
1196  allen-p/all_documents/756.   

                                                message  
1196  Message-ID: <12635597.1075855702772.JavaMail.e...  
DONE
File: allen-p/all_documents/756.
Message ID: <12635597.1075855702772.JavaMail.evans@thyme>
Subject: Re: Carlsbad/El Paso: Aug 23 Update: Press Conference scheduled on
 Thursday
From: phillip.allen@enron.com
To: scott.tholan@enron.com
Date: 1980-01-01 00:00:00+00:00

Body:
Scott,

 Thanks for the email.  I have two questions:

 1.  The 


In [18]:
from langchain_core.utils.function_calling import convert_to_openai_tool
from email_search_tools import search_emails, read_email

tool = convert_to_openai_tool(search_emails)
del tool["function"]["parameters"]["properties"]["inbox"]
tool["function"]["parameters"]["required"].remove("inbox")
tool
# convert_to_openai_tool(read_email)

{'type': 'function',
 'function': {'name': 'search_emails',
  'description': 'Searches the email database based on keywords, inbox, sender, recipient, and date range.',
  'parameters': {'properties': {'keywords': {'description': 'A list of keywords that must all appear in the subject or body.',
     'items': {'type': 'string'},
     'type': 'array'},
    'from_addr': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
     'default': None,
     'description': 'Optional email address to filter emails sent *from*.'},
    'to_addr': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
     'default': None,
     'description': 'Optional email address to filter emails sent *to* (inc. cc/bcc).'},
    'sent_after': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
     'default': None,
     'description': "Optional date string 'YYYY-MM-DD'. Filters for emails sent on or after this date."},
    'sent_before': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
     'default': None,
     'description'